In [98]:
import pandas as pd 
import numpy as np 

import os 
import sys 

from glob import glob 
from tqdm import tqdm 

import geopandas as gpd 
import matplotlib.pyplot as plt 

# enable latex plotting 
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

from shapely import wkt

In [99]:
og_points = pd.read_csv("/share/ju/urban-fingerprinting/notebooks/street_flooding/alt_evaluation/analysis_set.csv")

In [100]:
entire_sep29 = pd.read_csv("../notebooks/cambrian/entire_sep29_all.csv", engine='pyarrow')
entire_sep29['frame_id'] = entire_sep29['image_path'].apply(lambda x: x.split('/')[-1].split('.')[0])
entire_sep29

sep29_md = pd.read_csv("/share/ju/urban-fingerprinting/output/default/df/2023-09-29/md.csv", engine='pyarrow')
sep29_md['frame_id'] = sep29_md['frame_id'].apply(lambda x: "nlbx_"+x)

entire_sep29 = entire_sep29.merge(sep29_md, on='frame_id', how='left')


In [101]:
entire_sep29 = gpd.GeoDataFrame(entire_sep29, geometry=wkt.loads(entire_sep29['geometry']), crs='EPSG:2263')

In [102]:
sep29_positives = entire_sep29[entire_sep29['sentiment_1'] == 1]

In [103]:
analysis_set = pd.read_csv("../data/processed/flooding_ct_dataset.csv")

In [104]:
inferred_p_y = pd.read_csv("../data/processed/inferred_p_y.csv")

In [105]:
analysis_set['inferred_p_y'] = inferred_p_y['inferred_p_y']

In [106]:
nyc_ct = gpd.read_file('/share/ju/urban-fingerprinting/data/geo/nyc_ct/nyct2020.shp')
nyc_ct = nyc_ct.to_crs(2263)

In [107]:
analysis_set = gpd.GeoDataFrame(analysis_set, geometry=analysis_set.geometry.apply(lambda x: wkt.loads(x)), crs='EPSG:2263')
analysis_set

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,PUMA,Shape_Leng,Shape_Area,geometry,n_total,n_classified_positive,n_annotated_positive,n_annotated_negative,inferred_p_y
0,1.00,1,Manhattan,100,1000100,0,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,4121,11023.05,1844421.13,"MULTIPOLYGON (((972081.788 190733.467, 972184....",0.00,0.00,0.00,0.00,0.00
1,2.01,1,Manhattan,201,1000201,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4103,8010.31,1887578.27,"POLYGON ((988012.66 196152.484, 987984.769 196...",320.00,0.00,0.00,0.00,0.00
2,6.00,1,Manhattan,600,1000600,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,4103,9372.02,4429330.86,"POLYGON ((986961.185 199553.643, 987206.139 19...",845.00,2.00,1.00,1.00,0.00
3,14.01,1,Manhattan,1401,1001401,0,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,4103,5075.33,1006116.48,"POLYGON ((987382.076 199473.76, 987206.139 199...",182.00,0.00,0.00,0.00,0.00
4,14.02,1,Manhattan,1402,1001402,0,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4103,4459.16,1226206.20,"POLYGON ((988536.642 200278.524, 988404.814 19...",771.00,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,176.00,5,Staten Island,17600,5017600,0,Annadale-Huguenot-Prince's Bay-Woodrow,SI0304,SI03,SI03 South Shore (CD 3 Approximation),36085017600,4503,33259.44,58208511.41,"POLYGON ((938074.526 134571.59, 938187.252 134...",76.00,0.00,0.00,0.00,0.00
2323,228.02,5,Staten Island,22802,5022802,0,Freshkills Park (North),SI0291,SI02,SI02 Mid-Island (CD 2 Approximation),36085022802,4502,44266.03,68153751.49,"POLYGON ((938205.274 154530.769, 937703.178 15...",204.00,1.00,0.00,0.00,0.00
2324,291.02,5,Staten Island,29102,5029102,0,New Springville-Willowbrook-Bulls Head-Travis,SI0204,SI02,SI02 Mid-Island (CD 2 Approximation),36085029102,4502,55873.34,117574937.43,"POLYGON ((932410.624 168847.921, 932435.091 16...",1264.00,0.00,0.00,1.00,0.00
2325,161.00,2,Bronx,16100,2016100,0,Crotona Park East,BX0303,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005016100,4263,6476.48,2574283.84,"POLYGON ((1015839.339 245295.043, 1016167.845 ...",264.00,0.00,0.00,0.00,NaN


In [108]:
analysis_set = gpd.sjoin_nearest(analysis_set, nyc_ct, distance_col='nearest_frame_to_ct', lsuffix='', rsuffix='ct')
# drop index_left, index_right 
analysis_set.drop(columns=['index_ct'], inplace=True)

In [109]:
analysis_set

,CTLabel_,BoroCode_,BoroName_,CT2020_,BoroCT2020_,CDEligibil_,NTAName_,NTA2020_,CDTA2020_,CDTANAME_,...,CDEligibil_ct,NTAName_ct,NTA2020_ct,CDTA2020_ct,CDTANAME_ct,GEOID_ct,PUMA_ct,Shape_Leng_ct,Shape_Area_ct,nearest_frame_to_ct
0,1.00,1,Manhattan,100,1000100,0,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),...,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,4121,10833.04,1843004.52,0.00
1,2.01,1,Manhattan,201,1000201,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,None,Downtown Brooklyn-DUMBO-Boerum Hill,BK0202,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,36047002100,4302,12735.69,5707270.12,0.00
1,2.01,1,Manhattan,201,1000201,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,4103,6976.29,2582705.16,0.00
1,2.01,1,Manhattan,201,1000201,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4103,4754.50,972312.14,0.00
1,2.01,1,Manhattan,201,1000201,0,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,4103,5075.33,1006116.58,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,163.00,2,Bronx,16300,2016300,0,Crotona Park,BX0391,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,...,None,Claremont Village-Claremont (East),BX0302,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005016500,4263,7530.48,2083442.10,0.00
2326,163.00,2,Bronx,16300,2016300,0,Crotona Park,BX0391,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,...,None,Crotona Park East,BX0303,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005015500,4263,5835.15,1230923.83,0.00
2326,163.00,2,Bronx,16300,2016300,0,Crotona Park,BX0391,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,...,None,Crotona Park East,BX0303,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005016100,4263,6476.68,2574142.85,0.00
2326,163.00,2,Bronx,16300,2016300,0,Crotona Park,BX0391,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,...,None,Tremont,BX0602,BX06,BX06 Tremont-Belmont-West Farms (CD 6 Approxim...,36005036700,4263,4543.82,1005438.09,0.00


In [110]:
nyc_311 = pd.read_csv('/share/ju/urban-fingerprinting/data/geo/311_Service_Requests_from_2010_to_Present.csv', engine='pyarrow')

In [111]:
nyc_311['Created Date'] = pd.to_datetime(nyc_311['Created Date'])

In [112]:
# filter for sep 29 2023 complaints 
nyc_311 = nyc_311[nyc_311['Created Date'].dt.date == pd.to_datetime('2023-09-29').date()]

In [113]:
nyc_311 = gpd.GeoDataFrame(nyc_311, geometry=gpd.points_from_xy(nyc_311.Longitude, nyc_311.Latitude), crs='EPSG:4326')
nyc_311 = nyc_311.to_crs(2263)

In [114]:
flooding_descs = ["Street Flooding (SJ)", "Manhole Overflow (Use Comments) (SA1)", "Catch Basin Clogged/Flooding (Use Comments) (SC)"] 
nyc_311 = nyc_311[nyc_311['Descriptor'].isin(flooding_descs)]



In [115]:
len(nyc_311)

1048

In [116]:
# FLOODNET 
floodnet_sensor = pd.read_csv('/share/ju/urban-fingerprinting/data/nyc_flooding/floodnet-flood-sensor-sep-2023.csv', engine='pyarrow')
floodnet_tide = pd.read_csv('/share/ju/urban-fingerprinting/data/nyc_flooding/floodnet-tide-sep-2023.csv', engine='pyarrow')
floodnet_weather = pd.read_csv('/share/ju/urban-fingerprinting/data/nyc_flooding/floodnet-weather-sep-2023.csv', engine='pyarrow')


In [117]:
# how many unique sensors in each dataset
print("Floodnet Sensor", floodnet_sensor['deployment_id'].nunique())
print("Floodnet Tide", floodnet_tide['sensor_id'].nunique())
print("Floodnet Weather", floodnet_weather['sensor_id'].nunique())

Floodnet Sensor 67
Floodnet Tide 10
Floodnet Weather 16


In [118]:
all_floodnet_sensors_geo = pd.concat([floodnet_sensor.groupby('deployment_id').first()[['lat','lon']].reset_index(), floodnet_tide.groupby('sensor_id').first()[['lat','lon']].reset_index(), floodnet_weather.groupby('sensor_id').first()[['lat','lon']].reset_index()], axis=0)

all_floodnet_sensor_geo = gpd.GeoDataFrame(all_floodnet_sensors_geo, geometry=gpd.points_from_xy(all_floodnet_sensors_geo.lon, all_floodnet_sensors_geo.lat), crs='EPSG:4326').to_crs(2263)

In [119]:
# DEP STORMWATER 
MODERATE_CURRENT_CONDITIONS = '/share/ju/urban-fingerprinting/data/nyc_flooding/dep_stormwater_moderate_current/data.gdb'
moderate_current_conditions_layer = 'NYC_Stormwater_Flood_Map_Moderate_Flood_with_Current_Sea_Levels'

moderate_current_conditions = gpd.read_file(MODERATE_CURRENT_CONDITIONS, layer=moderate_current_conditions_layer).to_crs(2263)
moderate_current_conditions.describe()


,Flooding_Category,Shape_Length,Shape_Area
count,2.00,2.00,2.00
mean,1.50,4528920.47,60285414.57
std,0.71,2725631.39,26994084.32
min,1.00,2601608.03,41197714.49
25%,1.25,3565264.25,50741564.53
50%,1.50,4528920.47,60285414.57
75%,1.75,5492576.69,69829264.61
max,2.00,6456232.91,79373114.64


In [120]:
polygons = {} 
for i, row in moderate_current_conditions.iterrows():
    idx = 0
    for polygon in row['geometry'].geoms:
        polygons[f'{row["Flooding_Category"]}_{idx}'] = polygon
        idx += 1

# dataframe from dict 
stormwater_flattened = gpd.GeoDataFrame(polygons, index=['geometry']).T
# flooding category is the first part of the index


stormwater_flattened.set_geometry('geometry', inplace=True)
stormwater_flattened.crs = moderate_current_conditions.crs

stormwater_flattened['Flooding_Category'] = stormwater_flattened.index.str.split('_').str[0].astype(int)

In [121]:
acs_22_ct = pd.read_csv('/share/ju/urban-fingerprinting/notebooks/street_flooding/analysis/acs/ACSDP5Y2022.DP05-Data.csv')
# drop 1st row 
acs_22_ct = acs_22_ct.iloc[1:]
# remove US and everything prior from GEO_ID
acs_22_ct['GEO_ID'] = acs_22_ct['GEO_ID'].apply(lambda x: x.split('US')[1])


/tmp/ipykernel_2026539/3457170545.py:1: DtypeWarning: Columns (2,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,132,133,134,135,136,137,138,139,140,141,142,143,144,146,148,149,150,151,152,153,154,155,156,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,232,240,248,312,326,360) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_22_ct = pd.read_csv('/share/ju/urban-fingerprinting/notebooks/street_flooding/analysis/acs/ACSDP5Y2022.DP05-Data.csv')


In [122]:
nyc_ct = nyc_ct.merge(acs_22_ct, left_on='GEOID', right_on='GEO_ID', how='left')

In [123]:
TOTAL_POPULATION_ESTIMATE = 'DP05_0001E'

In [124]:
ct_enriched = nyc_ct.copy() 



ct_enriched.describe()

,Shape_Leng,Shape_Area,Unnamed: 366
count,2325.00,2325.00,0.00
mean,8451.42,3622740.96,NaN
std,7559.85,7163483.55,NaN
min,2569.48,400550.32,NaN
25%,5539.38,1631792.14,NaN
50%,6459.69,1964156.72,NaN
75%,8713.65,3087462.93,NaN
max,148864.82,197189857.74,NaN


In [125]:
ct_enriched[TOTAL_POPULATION_ESTIMATE] = ct_enriched[TOTAL_POPULATION_ESTIMATE].astype(float)
ct_enriched[TOTAL_POPULATION_ESTIMATE].describe()

count    2325.00
mean     3708.59
std      2059.97
min         0.00
25%      2305.00
50%      3447.00
75%      4876.00
max     15945.00
Name: DP05_0001E, dtype: float64

In [126]:
# get nearest 311 complaint to each tract 
ct_enriched = gpd.sjoin_nearest(ct_enriched, nyc_311, distance_col='nearest_report_to_ct')
# drop index_left, index_right, dont fail if they dont exist
ct_enriched.drop(columns=['index_right'], errors='ignore', inplace=True)

In [127]:
# get nearest floodnet sensor to each tract
ct_enriched = gpd.sjoin_nearest(ct_enriched, all_floodnet_sensor_geo, distance_col='nearest_sensor_to_ct')
# drop index_left, index_right, dont fail if they dont exist
ct_enriched.drop(columns=['index_right'], errors='ignore', inplace=True)

In [128]:
# get n_sensors per tract 
n_sensors_per_tract = all_floodnet_sensor_geo.sjoin(nyc_ct).groupby('GEOID').size().reset_index(name='n_sensors')
# merge into ct_enriched 
ct_enriched['n_sensors'] = ct_enriched['GEOID'].map(n_sensors_per_tract.set_index('GEOID')['n_sensors']).fillna(0)
print(ct_enriched['n_sensors'].describe())

count   3010.00
mean       0.19
std        0.82
min        0.00
25%        0.00
50%        0.00
75%        0.00
max        6.00
Name: n_sensors, dtype: float64


In [129]:
print(ct_enriched['n_sensors'].sum())

571.0


In [130]:
ct_enriched['nearest_sensor_to_ct'].describe()

count    3010.00
mean     6217.26
std      4856.31
min         0.00
25%      2448.78
50%      5366.14
75%      9018.49
max     28187.07
Name: nearest_sensor_to_ct, dtype: float64

In [131]:
# get nearest '1' flooding area to each tract
stormwater_filter = moderate_current_conditions['Flooding_Category'] == 1
ct_enriched = gpd.sjoin_nearest(ct_enriched, moderate_current_conditions[stormwater_filter], distance_col='nearest_nuisance_flooding_area_to_ct')
# drop index_left, index_right, dont fail if they dont exist
ct_enriched.drop(columns=['index_right'], errors='ignore', inplace=True)

# get nearest '2' flooding area to each tract
stormwater_filter = moderate_current_conditions['Flooding_Category'] == 2
ct_enriched = gpd.sjoin_nearest(ct_enriched, moderate_current_conditions[stormwater_filter], distance_col='nearest_deep_flooding_area_to_ct')
# drop index_left, index_right, dont fail if they dont exist
ct_enriched.drop(columns=['index_right'], errors='ignore', inplace=True)


In [132]:
ct_enriched['nearest_nuisance_flooding_area_to_ct'].describe()

count    3010.00
mean      341.00
std       773.05
min         0.00
25%         0.00
50%         0.00
75%       406.90
max     11490.93
Name: nearest_nuisance_flooding_area_to_ct, dtype: float64

In [133]:

# drop duplicates on GEOID 
ct_enriched.drop_duplicates(subset='GEOID', inplace=True)

In [134]:
nyc_311 = gpd.sjoin_nearest(nyc_311, nyc_ct, distance_col='nearest_complaint_to_ct')
# drop index_right 
nyc_311.drop(columns=['index_right'], inplace=True)

# count complaints per ct
ct_enriched['n_complaints'] = ct_enriched['CTLabel'].map(nyc_311.groupby('CTLabel').size()).fillna(0)


In [135]:
analysis_set['CTLabel_'] = analysis_set['CTLabel_'].apply(lambda x: str(x))
inferred_p_y_by_ct = analysis_set.groupby('CTLabel_')['inferred_p_y'].mean().reset_index()
# merge with ct_enriched
ct_enriched = ct_enriched.merge(inferred_p_y_by_ct, left_on='CTLabel', right_on='CTLabel_', how='left')
# fill na with 0
ct_enriched['inferred_p_y'] = ct_enriched['inferred_p_y'].fillna(0)


# count frames per ct 
TOP_N_TO_SELECT = 393
# if inferred_p_y is in the top N, then mark classified_positive as 1. else 0 
ct_enriched['classified_positive'] = ct_enriched['inferred_p_y'].rank(ascending=False, method='first') <= TOP_N_TO_SELECT

In [136]:
# get total number of 311 complaints in ct_enriched
total_complaints = nyc_311.shape[0]
print(total_complaints)

1037


In [137]:
stormwater_flattened = stormwater_flattened.overlay(ct_enriched, how='intersection')
stormwater_flattened['area'] = stormwater_flattened['geometry'].area
print(stormwater_flattened['area'].describe())
# convert area from sqft to sqmiles 
stormwater_flattened['area_sqmi'] = stormwater_flattened['area'] / 27878400
print(stormwater_flattened['area_sqmi'].describe())
ct_enriched['sqft_moderate'] = ct_enriched['CTLabel'].map(stormwater_flattened[stormwater_flattened['Flooding_Category'] == 1].groupby('CTLabel').area.sum()).fillna(0)
ct_enriched['sqft_deep'] = ct_enriched['CTLabel'].map(stormwater_flattened[stormwater_flattened['Flooding_Category'] == 2].groupby('CTLabel').area.sum()).fillna(0)

ct_enriched['sqmi_moderate'] = ct_enriched['CTLabel'].map(stormwater_flattened[stormwater_flattened['Flooding_Category'] == 1].groupby('CTLabel').area_sqmi.sum()).fillna(0)
ct_enriched['sqmi_deep'] = ct_enriched['CTLabel'].map(stormwater_flattened[stormwater_flattened['Flooding_Category'] == 2].groupby('CTLabel').area_sqmi.sum()).fillna(0)

# normalize as % of CT area 
#ct_enriched['sqft_moderate'] = ct_enriched['sqft_moderate'] / ct_enriched.geometry.area
#ct_enriched['sqft_deep'] = ct_enriched['sqft_deep'] / ct_enriched.geometry.area

# Set display options to not use scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# describe sqft_moderate, no scientific notation
print(ct_enriched['sqft_moderate'].describe())
print(ct_enriched['sqft_deep'].describe())

# describe sqmi_moderate, no scientific notation
print(ct_enriched['sqmi_moderate'].describe())
print(ct_enriched['sqmi_deep'].describe())

count    27280.00
mean      4418.20
std      11795.17
min          0.00
25%        259.10
50%        681.04
75%       3421.25
max     327700.14
Name: area, dtype: float64
count   27280.00
mean        0.00
std         0.00
min         0.00
25%         0.00
50%         0.00
75%         0.00
max         0.01
Name: area_sqmi, dtype: float64
count     2325.00
mean     64936.56
std     110309.69
min          0.00
25%          0.11
50%      24188.31
75%      75559.26
max     996037.01
Name: sqft_moderate, dtype: float64
count     2325.00
mean     34708.70
std      72915.18
min          0.00
25%          0.00
50%       5933.33
75%      34388.53
max     737821.33
Name: sqft_deep, dtype: float64
count   2325.00
mean       0.00
std        0.00
min        0.00
25%        0.00
50%        0.00
75%        0.00
max        0.04
Name: sqmi_moderate, dtype: float64
count   2325.00
mean       0.00
std        0.00
min        0.00
25%        0.00
50%        0.00
75%        0.00
max        0.03
Name: sqmi_de

In [138]:
# drop duplicate rows on CTLabel
ct_enriched = ct_enriched.drop_duplicates(subset='GEOID')

In [139]:
ct_enriched

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,...,Shape_Area,nearest_deep_flooding_area_to_ct,n_complaints,CTLabel_,inferred_p_y,classified_positive,sqft_moderate,sqft_deep,sqmi_moderate,sqmi_deep
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),...,41197714.49,7330.15,0.00,NaN,0.00,False,0.00,0.00,0.00,0.00
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,41197714.49,830.90,0.00,2.01,0.00,False,0.00,0.00,0.00,0.00
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,41197714.49,0.00,2.00,NaN,0.00,False,278099.84,125570.17,0.01,0.00
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,41197714.49,587.41,0.00,14.01,0.00,False,0.00,0.00,0.00,0.00
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),...,41197714.49,0.00,0.00,14.02,0.00,False,3918.73,3842.66,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,98,1,Manhattan,009800,1009800,None,East Midtown-Turtle Bay,MN0604,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,...,41197714.49,0.00,0.00,NaN,0.00,False,73287.85,51665.54,0.00,0.00
2321,86.03,1,Manhattan,008603,1008603,None,East Midtown-Turtle Bay,MN0604,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,...,41197714.49,757.66,0.00,86.03,0.00,False,0.00,0.00,0.00,0.00
2322,90,1,Manhattan,009000,1009000,None,East Midtown-Turtle Bay,MN0604,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,...,41197714.49,0.00,2.00,NaN,0.00,False,6960.22,649.24,0.00,0.00
2323,319.01,5,Staten Island,031901,5031901,None,Mariner's Harbor-Arlington-Graniteville,SI0107,SI01,SI01 North Shore (CD 1 Equivalent),...,41197714.49,0.00,0.00,319.01,0.00,True,11704.88,6993.95,0.00,0.00


In [140]:
# decsribe distribution of tracts with no complaints, nearest_complaint_to_ct
ct_enriched['nearest_report_to_ct'].describe()

count   2325.00
mean     933.23
std     1156.99
min        0.00
25%        0.47
50%      566.21
75%     1371.32
max     7945.05
Name: nearest_report_to_ct, dtype: float64

In [141]:
ct_enriched.columns.values

array(['CTLabel', 'BoroCode', 'BoroName', 'CT2020', 'BoroCT2020',
       'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME',
       'GEOID', 'PUMA', 'Shape_Leng', 'Shape_Area_left', 'geometry',
       'GEO_ID', 'NAME', 'DP05_0001E', 'DP05_0001M', 'DP05_0002E',
       'DP05_0002M', 'DP05_0003E', 'DP05_0003M', 'DP05_0004E',
       'DP05_0004M', 'DP05_0005E', 'DP05_0005M', 'DP05_0006E',
       'DP05_0006M', 'DP05_0007E', 'DP05_0007M', 'DP05_0008E',
       'DP05_0008M', 'DP05_0009E', 'DP05_0009M', 'DP05_0010E',
       'DP05_0010M', 'DP05_0011E', 'DP05_0011M', 'DP05_0012E',
       'DP05_0012M', 'DP05_0013E', 'DP05_0013M', 'DP05_0014E',
       'DP05_0014M', 'DP05_0015E', 'DP05_0015M', 'DP05_0016E',
       'DP05_0016M', 'DP05_0017E', 'DP05_0017M', 'DP05_0018E',
       'DP05_0018M', 'DP05_0019E', 'DP05_0019M', 'DP05_0020E',
       'DP05_0020M', 'DP05_0021E', 'DP05_0021M', 'DP05_0022E',
       'DP05_0022M', 'DP05_0023E', 'DP05_0023M', 'DP05_0024E',
       'DP05_0024M', 'DP05_0025E', 'DP

In [142]:
ct_enriched['classified_positive'].isna().sum()

0

In [143]:
ct_enriched['classified_positive'].value_counts()

classified_positive
False    1932
True      393
Name: count, dtype: int64

In [144]:
# how many people live in tracts with n_frames > 0 but no 311 report? 
print(ct_enriched[(ct_enriched['classified_positive']) & (ct_enriched['n_complaints'] == 0)][TOTAL_POPULATION_ESTIMATE].sum())

972626.0


In [145]:
# how many people live in tracts with n_frames > 0 but no floodnet sensor? 
print(ct_enriched[(ct_enriched['classified_positive']) & (ct_enriched['n_sensors'] == 0)][TOTAL_POPULATION_ESTIMATE].sum())


1277804.0


In [146]:
# how many people live in tracts with n_frames > 0 but no sqmi of moderate or deep flooding? 
print(ct_enriched[(ct_enriched['classified_positive']) & (ct_enriched['sqft_moderate'] > 0) & (ct_enriched['sqft_deep'] > 0)][TOTAL_POPULATION_ESTIMATE].sum())

702595.0


In [147]:
THRESHOLDS = np.linspace(0.0, 1, num=100)
MULTIPLIER = 5280 

pr_stats = {

}

for threshold in THRESHOLDS:
    pr_stats[threshold] = { 
        'tracts_with_reports_within_threshold': ct_enriched[ct_enriched['nearest_report_to_ct'] <= threshold * MULTIPLIER].shape[0],
        'tracts_with_sensors_within_threshold': ct_enriched[ct_enriched['nearest_sensor_to_ct'] <= threshold * MULTIPLIER].shape[0],
        'tracts_with_nuisance_flooding_within_threshold': ct_enriched[ct_enriched['nearest_nuisance_flooding_area_to_ct'] <= threshold * MULTIPLIER].shape[0],
        'tracts_with_deep_flooding_within_threshold': ct_enriched[ct_enriched['nearest_deep_flooding_area_to_ct'] <= threshold * MULTIPLIER].shape[0],
        'tracts_with_all_within_threshold': ct_enriched[(ct_enriched['nearest_report_to_ct'] <= threshold * MULTIPLIER) & (ct_enriched['nearest_sensor_to_ct'] <= threshold * MULTIPLIER) & (ct_enriched['nearest_nuisance_flooding_area_to_ct'] <= threshold * MULTIPLIER) & (ct_enriched['nearest_deep_flooding_area_to_ct'] <= threshold * MULTIPLIER)].shape[0]
    }
    

pr_stats = pd.DataFrame(pr_stats).T
pr_stats

,tracts_with_reports_within_threshold,tracts_with_sensors_within_threshold,tracts_with_nuisance_flooding_within_threshold,tracts_with_deep_flooding_within_threshold,tracts_with_all_within_threshold
0.00,547,60,1324,1097,25
0.01,704,73,1385,1195,32
0.02,733,78,1430,1252,38
0.03,766,82,1481,1303,39
0.04,799,87,1534,1361,45
...,...,...,...,...,...
0.96,2293,1037,2321,2320,1029
0.97,2294,1047,2321,2320,1038
0.98,2295,1053,2321,2320,1045
0.99,2297,1059,2321,2320,1051
